# Desenvolupament pràctic TFG

Per al desenvolupament del projecte pràctic, farem us de una base de dades que les seves entrades consisteixen en una persona que demana un crèdit al banc. Cada persona es classifica segons el risc que generi fer-li un prèstam (poden ser bons prestams o dolents).

In [22]:
import pandas as pd
import altair as alt
from IPython.display import display
import warnings

warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Anàlisi de les dades

Primer de tot, haurem de carregar les dades en un fitxer.

In [2]:
data = pd.DataFrame(pd.read_csv("./archive/german_credit_data.csv")).drop("Id", axis=1)
data

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad


Com podem veure en el display anterior, tenim un total de 1000 files (sent cada fila uan persona) i cada una de les files compten amb 10 columnes.

In [3]:
# unique to extract values

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               1000 non-null   int64 
 1   Sex               1000 non-null   object
 2   Job               1000 non-null   int64 
 3   Housing           1000 non-null   object
 4   Saving accounts   817 non-null    object
 5   Checking account  606 non-null    object
 6   Credit amount     1000 non-null   int64 
 7   Duration          1000 non-null   int64 
 8   Purpose           1000 non-null   object
 9   Risk              1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 78.2+ KB


Podem veure en el resum de les dades les diferents columnes que té el dataset. Les columnes son les següents:

1. Age: consisteix en la edat de la persona, de tipus enter
2. Sex: consisteix en el genere de la persona, el qual es defineix amb un string
3. Job: Consisteix en una classificació segons les hbailitats de la persona:
    - 0: no té habilitat i no és resident
    - 1: no té habilitat i és resident
    - 2: té habilitat
    - 3: és molt habilidós
4. Housing: consisteix en la propietat de vivenda de la persona:
    - free
    - rent
    - own
5. Saving accounts: consisteix en un nivell d'estalvi:
    - Little
    - Moderate
    - Rich
    - Quite Rich
6. Checking accounts: 
    - Little
    - Moderate
    - Rich
    - Quite Rich
7. Credit Amount: quantitat de credit, de tipus enter
8. Duration: duració del credit en mesos
9. Purpose: indica el motiu per el qual es demana el credit, en forma text
10. Risk: indica si el risc de donar el crèdit es bo o no.

Un altre cosa que podem veure es que la llista compte amb valors nulls (Nan) en les columnes Savings accounts i Checking accounts, per la qual cosa, a aquesta valors nans els substituirem per una nova categoria la qual s'anomenarà "unknown".

In [5]:
data.fillna(value="unknown", inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               1000 non-null   int64 
 1   Sex               1000 non-null   object
 2   Job               1000 non-null   int64 
 3   Housing           1000 non-null   object
 4   Saving accounts   1000 non-null   object
 5   Checking account  1000 non-null   object
 6   Credit amount     1000 non-null   int64 
 7   Duration          1000 non-null   int64 
 8   Purpose           1000 non-null   object
 9   Risk              1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 78.2+ KB


Ara podem observar com totes les files amb valors Nan ja no tenen aquests valors, ja que s'ha omplert amb el valor "unknown".

Per començar amb l'anàlisi de les dades, mostrarem un seguit de gràfics interessants que ens permetin entrendre millor les dades.

In [6]:
# bins = [0, 10, 20, 30, 60]
# labels = ["Infant", "Kid", "Young", "Adult"]

# dataset_ages_class = data["Age"].reset_index(name="Age")
# dataset_ages_class["AgeGroup"] = pd.cut(dataset_ages_class["Age"], 
#                                         bins=bins, 
#                                         labels=labels, 
#                                         right=False)

dataset = data.groupby("Age").size().reset_index(name="CountPeople")
alt.Chart(dataset).mark_bar().encode(
    x=alt.X("Age", title="Age", sort=None),
    y=alt.Y("CountPeople", title="Count of people"))


alt.Chart(...)

Podem veure en aquest gràfic la quantitat de persones que hi ha per cada edat. Veiem que la gran majoria de persones que demanen un crèdit al banc és gent jove, la qual es concenctra especialment a al franja dels 23 anys fins els 36. 

Tenint en compte això, pot ser interessant veure els principals motius que troben les persones entre aquesta franja d'edat abans mencionada.

In [7]:
dataset_young = data[data.Age > 19]
dataset_young = dataset_young[dataset_young.Age < 40]

dataset_young = dataset_young.groupby("Purpose").size().reset_index(name="Count")
alt.Chart(dataset_young).mark_bar().encode(
    y=alt.Y("Purpose", title="Different purposes", sort=None),
    x=alt.X("Count", title="Count of people by purpose", sort=None))

alt.Chart(...)

Els resultats obtinguts poden resultar en certa forma sorprenents, ja que en aquestes edats, la gran majoria de persones solen demanar un crèdit per a poder afrontar el pagament de una vivenda o potser afrontar uns estudis o qualsevol altre tipus de formació, però com mostra la gràfica, els motius més comuns en aquesta franja són:

1. Compra d'un cotxe
2. Compra de una radio o televisió
3. Mobles i equipació.

Un altre factor que podem analitzar de cara a la gent jove és veure una comparació de riquesa entre les diferents generacions: per això, separarem les dades entre usuaris amb edat igual o menor a 50 anys i usuaris amb més de 50 anys. D'aquesta manera, podrem identificar si les persones amb més edat dispossen en general de una millor situació economica que els joves.

In [8]:
dataset_less_50 = data[data.Age <= 50]
dataset_more_50 = data[data.Age > 50]

savings_less_50 = dataset_less_50["Saving accounts"].value_counts().reset_index(name="PersonCount")
savings_more_50 = dataset_more_50["Saving accounts"].value_counts().reset_index(name="PersonCount")

savings_less_50["Percentage"] = round((savings_less_50["PersonCount"] * 100)  / len(dataset_less_50), 2)
savings_more_50["Percentage"] = round((savings_more_50["PersonCount"] * 100)  / len(dataset_less_50), 2)

alt.Chart(savings_less_50).mark_bar().encode(
    y=alt.Y("index", title="Different purposes", sort=None),
    x=alt.X("PersonCount", title="Count of people by purpose", sort=None))

alt.Chart(...)

In [9]:
# alt.Chart(savings_more_50).mark_bar().encode(
#     y=alt.Y("index", title="Different purposes", sort=None),
#     x=alt.X("count", title="Count of people by purpose", sort=None))

Si veiem els dos gràfics, novament podem observar com a partir dels 50 anys, el número de persones que demanen un crèdit es redueix dràsticament. De totes formes, els gràfics constaten que en els dos casos, la gradissima majoria de les persones compten amb un nivell d'estalvi classificat com a petit. 


### Entrenament per al model

Després d'haver realitzat un analisi de les dades, haurem de preparar les dades per a l'entrenament del model.


In [2]:
data = pd.DataFrame(pd.read_csv("./archive/german_credit_data.csv")).drop("Id", axis=1)
data.isnull().sum().sort_values(ascending=False)

Checking account    394
Saving accounts     183
Age                   0
Sex                   0
Job                   0
Housing               0
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

Com podem veure a la cel·la anterior, a la columna de Checking account comptem amb 394 files amb valors Nan i Saving accounts compte amb 183 valors Nan.

Donat que la llibreria Carla no pot tractar amb dades que continguin valors Nan, haurem de fer un tractament de les dades a fi de otorgar un format acceptat. La primera opció que tenim per fer aquesta feina es buidar aquelles fileres que no tinguin un valor Nan.

In [3]:
data_no_nan = data.dropna()
print("Dataset size after droping Nan values:", data_no_nan.shape)

Dataset size after droping Nan values: (522, 10)


Com podem veure a la cel·la anterior, el tamany del dataset es veu molt reduït, passant de 1000 files a 522.

Donat que això pot reduir molt la qualitat de l'estudi, no podem considerar suficientment acceptable la qualitat del dataset amb una reducció tan gran de les dades (quasi un 50% de les dades han desaparegut).

Per això, aprofitant que en el punt anterior hem intercanviat els valors Nan per el valor "unkown", aprofitarem el mateix dataset amb aquests valors modificats a fi de poder aprofitar el 100% de les files.

In [4]:
data.fillna(value="unknown", inplace=True)
data_carla = data

for i in range(len(data_carla["Risk"])):
    if data_carla["Risk"][i] == 'good':
        data_carla["Risk"][i] = 1.0
    else:
        data_carla["Risk"][i] = 0.0

data_carla.info()
data_carla.to_csv("./archive/german_credit_data_noNan.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               1000 non-null   int64 
 1   Sex               1000 non-null   object
 2   Job               1000 non-null   int64 
 3   Housing           1000 non-null   object
 4   Saving accounts   1000 non-null   object
 5   Checking account  1000 non-null   object
 6   Credit amount     1000 non-null   int64 
 7   Duration          1000 non-null   int64 
 8   Purpose           1000 non-null   object
 9   Risk              1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 78.2+ KB


Ara si podem començar a usar les dades amb la llibreria Carla. Primer de tot, haurem de fer un tractament d'aquestes dades per a que siguin compatibles amb carla, fent us de la funció *CsvCatalog*.

In [118]:
from carla.data.catalog import CsvCatalog
from carla.models.catalog.catalog import OnlineCatalog

continuous = ["Age", "Credit amount", "Duration", "Job"]
categorical = [ "Sex", "Housing", "Saving accounts", "Checking account", "Purpose"]
immutable = []
data_bank = CsvCatalog(file_path = "./archive/german_credit_data_noNan.csv",
                 continuous=continuous,
                 categorical=categorical,
                 immutables=immutable,
                 target='Risk')

display(data_bank.df)

,Age,Job,Credit amount,Duration,Risk,...,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
0,0.857143,0.666667,0.050567,0.029412,1.0,...,0.0,0.0,1.0,0.0,0.0
1,0.053571,0.666667,0.313690,0.647059,0.0,...,0.0,0.0,1.0,0.0,0.0
2,0.535714,0.333333,0.101574,0.117647,1.0,...,1.0,0.0,0.0,0.0,0.0
3,0.464286,0.666667,0.419941,0.558824,1.0,...,0.0,1.0,0.0,0.0,0.0
4,0.607143,0.666667,0.254209,0.294118,0.0,...,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
995,0.214286,0.333333,0.081765,0.117647,1.0,...,0.0,1.0,0.0,0.0,0.0
996,0.375000,1.000000,0.198470,0.382353,1.0,...,0.0,0.0,0.0,0.0,0.0
997,0.339286,0.666667,0.030483,0.117647,1.0,...,0.0,0.0,1.0,0.0,0.0
998,0.071429,0.666667,0.087763,0.602941,0.0,...,0.0,0.0,1.0,0.0,0.0


Un cop tenim les dades preparades, haurem de preparar el nostre model per a poder-la entrenar.

In [119]:
from carla.models.catalog.catalog import MLModelCatalog

# Paramos for training
training_params = {"lr": 0.002, "epochs": 10, 
                   "batch_size": 1024, "hidden_size": [18, 9, 3], 
                   "max_depth": 10}

model = MLModelCatalog(data=data_bank,
                      model_type="forest",
                      backend="xgboost",
                      load_online=False)

model.train(learning_rate=training_params["lr"],
            epochs=training_params["epochs"],
            batch_size=training_params["batch_size"],
            hidden_size=training_params["hidden_size"],
            max_depth=training_params["max_depth"])

display(model.data.df_test)

balance on test set 0.6813333333333333, balance on test set 0.756
[0]	validation_0-logloss:0.59013	validation_1-logloss:0.63664
[1]	validation_0-logloss:0.52363	validation_1-logloss:0.60372
[2]	validation_0-logloss:0.47543	validation_1-logloss:0.58615
[3]	validation_0-logloss:0.43667	validation_1-logloss:0.57232
[4]	validation_0-logloss:0.40703	validation_1-logloss:0.56463


,Age,Job,Credit amount,Duration,Risk,...,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
263,0.678571,0.333333,0.137449,0.117647,1.0,...,1.0,0.0,0.0,0.0,0.0
389,0.142857,0.666667,0.065973,0.073529,1.0,...,0.0,0.0,0.0,0.0,0.0
657,0.321429,0.666667,0.548696,0.647059,1.0,...,0.0,0.0,1.0,0.0,0.0
867,0.410714,0.666667,0.169528,0.117647,1.0,...,0.0,1.0,0.0,0.0,0.0
800,0.625000,0.666667,0.074117,0.294118,1.0,...,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
711,0.285714,0.666667,0.052162,0.029412,0.0,...,1.0,0.0,0.0,0.0,0.0
182,0.375000,0.333333,0.076868,0.250000,0.0,...,0.0,0.0,0.0,0.0,0.0
756,0.982143,0.000000,0.057720,0.029412,1.0,...,0.0,0.0,0.0,0.0,0.0
206,0.428571,0.666667,0.092715,0.117647,1.0,...,0.0,1.0,0.0,0.0,0.0


In [126]:
list_predictions = model.predict(model.data.df_test)
display(list_predictions)

array([1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 1., 0.

In [125]:
data = [["90", "male", "1", "rent", "little", "little", 0, 8, "furniture/equipment"]]
columns = ["Age","Sex","Job","Housing","Saving accounts",
           "Checking account","Credit amount","Duration","Purpose"]

random_person_to_predict = pd.DataFrame(data=data, columns=columns)

random_value = model.predict(random_person_to_predict)

display(random_value)

array([1.])

En aquest moment, tenim un model de tipus ann el qual es troba entrenat, i com retorna el seu valor, 

In [42]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals =  predict_negative_instances(model, data_bank.df)
test_factual = factuals

value = model.predict(x=test_factual)